# How to use chat models to call tools

1. Tool Schema Definition

Tools need schemas that describe their functionality and argument structure. You can define these schemas using Python functions, Pydantic models, or TypedDicts.

# Using Python Functions

In [6]:
# The function name, type hints, and docstring are all part of the tool
# schema that's passed to the model. Defining good, descriptive schemas
# is an extension of prompt engineering and is an important part of
# getting models to perform well.
def add(a: int, b: int) -> int:
    """Add two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a + b


def multiply(a: int, b: int) -> int:
    """Multiply two integers.

    Args:
        a: First integer
        b: Second integer
    """
    return a * b

Explanation:

Function names represent tool names.

Type hints and docstrings provide argument and functionality details.


# Using Pydantic Models

In [5]:
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")

Explanation:

Pydantic provides validation.

Descriptions enhance tool schema clarity.

# Using TypedDicts

In [7]:
from typing_extensions import Annotated, TypedDict


class add(TypedDict):
    """Add two integers."""

    # Annotations must have the type and can optionally include a default value and description (in that order).
    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


class multiply(TypedDict):
    """Multiply two integers."""

    a: Annotated[int, ..., "First integer"]
    b: Annotated[int, ..., "Second integer"]


tools = [add, multiply]

Explanation:

TypedDicts are lightweight compared to Pydantic.

Annotations define argument details.

# 3. Binding Tools to Chat Models
After defining tools, bind them to a chat model using .bind_tools().

In [12]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [15]:
import getpass
import os

os.environ["GROQ_API_KEY"] = getpass.getpass()

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-8b-8192")

In [16]:
llm_with_tools = llm.bind_tools(tools)

query = "What is 3 * 12?"

llm_with_tools.invoke(query)

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_1ydm', 'function': {'arguments': '{"a":3,"b":12}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 79, 'prompt_tokens': 1064, 'total_tokens': 1143, 'completion_time': 0.065833333, 'prompt_time': 0.140211297, 'queue_time': 0.0030424989999999763, 'total_time': 0.20604463}, 'model_name': 'llama3-8b-8192', 'system_fingerprint': 'fp_6a6771ae9c', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-e12fad11-63b9-4409-bbb5-08cbf8b61668-0', tool_calls=[{'name': 'multiply', 'args': {'a': 3, 'b': 12}, 'id': 'call_1ydm', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1064, 'output_tokens': 79, 'total_tokens': 1143})

In [17]:
query = "What is 3 * 12? Also, what is 11 + 49?"

llm_with_tools.invoke(query).tool_calls

[{'name': 'multiply',
  'args': {'a': 3, 'b': 12},
  'id': 'call_e9zj',
  'type': 'tool_call'},
 {'name': 'add',
  'args': {'a': 11, 'b': 49},
  'id': 'call_077x',
  'type': 'tool_call'}]

Explanation:

The model generates structured arguments for the tools (e.g., {"a": 3, "b": 12}).

You decide whether to execute the tools or further process the arguments.


# 4. Tool Calls and Responses
A tool call object contains:

* Tool name
* Arguments (as JSON or a structured format)
* Optional metadata (e.g., IDs).

In [18]:
from langchain_core.output_parsers import PydanticToolsParser
from pydantic import BaseModel, Field


class add(BaseModel):
    """Add two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


class multiply(BaseModel):
    """Multiply two integers."""

    a: int = Field(..., description="First integer")
    b: int = Field(..., description="Second integer")


chain = llm_with_tools | PydanticToolsParser(tools=[add, multiply])
chain.invoke(query)

[multiply(a=3, b=12), add(a=11, b=49)]

# Summary
* Tool Schema: Define tools using Python functions, Pydantic models, or TypedDicts.
* Binding Tools: Attach tool schemas to the chat model using .bind_tools().
* Tool Calls: Generate structured arguments; decide execution.
* Output Parsing: Use parsers like PydanticToolsParser to validate and process results.